In [1]:
def categorical_crossentropy(t,y):
    return np.mean(-t*np.log(y+0.0001))

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x+0.001))

In [3]:
def softmax(x):
    c = np.max(x,axis=1).reshape(-1,1)
    x = x-c
    return np.exp(x)/np.sum(np.exp(x),axis=1).reshape(-1,1)

In [4]:
import numpy as np
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self,x):
        self.mask = (x <=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        return dx
        
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self,x):
        out = sigmoid(x)
        self.out = out
        return out
    
    def backward(self,dout):
        dx = ((1-self.out)*self.out)*dout
        return dx

class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.origin_shape = None
        self.dW = None
        self.db = None
    
    def forward(self,x):
        self.origin_shape = x.shape
        self.x = x
        out = np.dot(x,self.W) + self.b
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        dx = dx.reshape(self.origin_shape)
        return dx

class Loss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self,t,y):
        self.y = softmax(y)
        self.t = t
        self.loss = categorical_crossentropy(self.t, self.y)
        return self.loss
    
    def backward(self,dout=1):
        dx = (self.y - self.t)*dout
        return dx

In [5]:
class Layers:
    def __init__(self):
        self.layers = {}
        
    def add(self,x1,x2,activation):
        activation_dict = {
            'sigmoid':Sigmoid,
            'relu':Relu,
            'softmax':Loss
        }
        w = np.random.randn(x1,x2)
        b = np.zeros(x2)
        activation_layer = 'activation'+str((int(len(self.layers)/2+1))) 
        Affine_layer = 'Affine'+str((int(len(self.layers)/2+1)))
        self.layers[Affine_layer] = Affine(w,b)
        self.layers[activation_layer] = activation_dict[activation]()
        
    
    def predict(self,x):
        out = x.copy()
        ind = 1 
        layer_len = len(self.layers)
        for key, layer in self.layers.items():
            if ind < layer_len :
                out = layer.forward(out)
            ind += 1
        return out
            
    
    def loss(self,x,t):
        y = self.predict(x)
        out = list(self.layers.values())[-1].forward(t,y)
        return out
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        self.acc = np.sum(y==t)/t.size
        return self.acc
    
    def gradient(self,x,t):
        self.loss(x,t)
        lr = 1e-4
        dout = 1
        dout = list(self.layers.values())[-1].backward(dout)
        layers = list(self.layers.values())[::-1][1:]
        self.layers_key = list(self.layers.keys())[::-1][1:]
        for layer in layers:
            dout = layer.backward(dout)
        self.grads = {}
        for layer_key in self.layers_key:
            if 'Affine' in layer_key:
                self.grads[layer_key] = [self.layers[layer_key].dW, self.layers[layer_key].db]
        for layer_key in self.layers_key:
            if 'Affine' in layer_key:
                self.layers[layer_key].W -= lr*self.grads[layer_key][0]
                self.layers[layer_key].b -= lr*self.grads[layer_key][1]
        result = self.loss(x,t)       
        return result
    
    def fit(self,x,t,epochs,lr):
        self.lr = lr
        self.history = {}
        accuracy = []
        loss = []
        for epoch in range(epochs):
            self.gradient(x,t)
            loss.append(self.err)
            accuracy.append(self.accuracy(x,t))
            if epoch % 100 == 0:
                print(f'loss : {self.err} === accuracy : {self.accuracy(x,t)}')
        self.history['accuracy'] = accuracy
        self.history['loss'] = loss
                

In [6]:
## 데이터 정리
## network 설계 x데이터의 shape, y데이터의 형태
## 모델학습

In [7]:
## 필요한 모듈 호출
import numpy as np
import tensorflow as tf
from tensorflow import keras
# 손글씨데이터 업로드
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

In [8]:
## 데이터 정리
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
## 데이터확인
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [10]:
# x데이터 변환
x_train = x_train.reshape(-1,28*28)
print(x_train.shape)

(60000, 784)


In [11]:
# y데이터 변환
y_train.shape
## 정수 인코딩 되어 있고
np.unique(y_train).size
## one-hot 인코딩이 필요
y_train = to_categorical(y_train)
y_train.shape ## (60000,10)

(60000, 10)

In [12]:
model = Layers()

In [13]:
model.add(784,2048,'relu')
model.add(2048,1024,'relu')
model.add(1024,512,'relu')
model.add(512,10,'softmax')

In [14]:
model.layers['Affine1'].W.shape

(784, 2048)

In [15]:
model.predict(x_train).shape

(60000, 10)

In [16]:
for i in range(10):
    print(model.gradient(x_train,y_train))

0.8301116400416148
0.8312168928856521
0.8301116400416148
0.3682632205063057
0.35321350822462605
0.33806614378196265
0.32298406987669476
0.3087065690882985
0.29616089268219703
0.2860949436668341


In [17]:
pred = model.predict(x_test.reshape(-1,28*28))

In [20]:
pred = np.argmax(pred,axis=1)

In [21]:
np.sum(pred == y_test)/y_test

C:\Users\admin\AppData\Local\Temp\ipykernel_6224\330435189.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.sum(pred == y_test)/y_test
C:\Users\admin\AppData\Local\Temp\ipykernel_6224\330435189.py:1: RuntimeWarning: invalid value encountered in divide
  np.sum(pred == y_test)/y_test


array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
categorical_crossentropy(pred,y_train)

0.8282182968056169